<a href="https://colab.research.google.com/github/jszyxw/oi-code-hub/blob/master/%E4%BA%92%E8%81%94%E7%BD%91%E6%95%B0%E6%8D%AE%E6%8C%96%E6%8E%98%E4%BD%9C%E4%B8%9A%E6%8A%A5%E5%91%8A_%E6%A8%A1%E5%9E%8B%E8%AE%AD%E7%BB%83%E9%83%A8%E5%88%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 互联网数据挖掘作业报告 模型训练部分
## Transformers & BiGRU 机器生成文本的自动识别（使用colab训练）

**1800013079 刘宇晗**

在这次作业中，我使用了 BERT (Bidirectional Encoder Representations from Transformers)模型作为编码器，来自于 [这篇论文](https://arxiv.org/abs/1810.04805). 由于**计算资源和时间的限制**：
1. 无法使用参数量更大的模型。在编码器方面使用了预训练的BERT(base)模型，且在训练模型时，不对BERT模型进行`fine-tune`训练
2. 整体模型结构：$BERT(not\ train, bert-base) \rightarrow BiGRU\times n \rightarrow FC output$，由数据分析部分可知，有四分之一左右的文本超出了bert-base的最大长度，因此使用长度更大的预训练模型会产生更好的效果（或使用其他在数据分析部分提到的解决方法），但由于算力的限制，使用更大的模型是不可行的（超出colab的12小时最大运行时间，加上3. 所述的中断连接问题的限制）
3. 在训练过程中，colab 多次出现中断的问题，浪费了我将近2天的时间，很多次在上传数据集时就断连，最好的一次也仅对模型训练了3个epoch（如本notebook存档所示），远未达到收敛，因此理论上模型应该还有进一步训练的空间。目前我的colab账号已经达到GPU的使用限额，无法再次训练，之后获得GPU限额后将再次训练测试结果。


## Preparing Data


In [ ]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

加载预训练的 `bert-base-uncased`模型的 tokenizer.

In [ ]:
!pip install transformers
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

     |████████████████████████████████| 1.3MB 5.4MB/s 
     |████████████████████████████████| 890kB 23.2MB/s 
     |████████████████████████████████| 2.9MB 36.2MB/s 
     |████████████████████████████████| 1.1MB 44.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=68095569e637774ee6161ac4d178ac42124636f7da1411583b82219622caf0d7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
len(tokenizer.vocab)

30522

In [ ]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


查看预训练模型定义的 `max_model_input_sizes`, 即 bert 的最大读入长度:

In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

定义读入训练集的fields

In [ ]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

读入数据

In [ ]:
from torchtext import datasets
import os

train_data, test_data = data.TabularDataset.splits( path = '/content/',
                            train = 'train.txt',
                            test = 'test1.txt',
                            format = 'tsv', #'tsv' for tabs, 'csv' for commas
                            fields = [('text',TEXT), ('label',LABEL)],
)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 69993
Number of validation examples: 29997
Number of testing examples: 9991


样例测试

In [ ]:
print(vars(train_data.examples[6]))

{'text': [2005, 1996, 2293, 1997, 11084, 2054, 3599, 2003, 2293, 1029, 2079, 2057, 2031, 2000, 2903, 1999, 2009, 2000, 2022, 9398, 10205, 1029, 5751, 1997, 22289, 6550, 29500, 10247, 2008, 17727, 12184, 5897, 2003, 2019, 17245, 2017, 1005, 2128, 11973, 1999, 1037, 22289, 3276, 2729, 3672, 8606, 23032, 2065, 2017, 2024, 2369, 1999, 2115, 2729, 23795, 1998, 2514, 2066, 2017, 1005, 2128, 4634, 2369, 4256, 2005, 4256, 2030, 1996, 3232, 2003, 27673, 2205, 4030, 1999, 1037, 3276, 1516, 4638, 2007, 2814, 1998, 5826, 2006, 2591, 2865, 2000, 2131, 15690, 2005, 3176, 4506, 2000, 2202, 2006, 1037, 2154, 1011, 2000, 1011, 2154, 3978, 6781, 1024, 4738, 3348, 19294, 10639, 2015, 2007, 5826, 2012, 3091, 2055, 4424, 1998, 3276, 9967, 2077, 2062, 10305, 4262, 4088, 3678, 7007, 6825, 4219, 5009, 1024, 7721, 21111, 3406, 10727, 29202, 26572, 22591, 13288, 4813, 2415, 12468, 2005, 2274, 2000, 2322, 8361, 4122, 5352, 9656, 1024, 2991, 7610, 2072, 19461, 1516, 2054, 2001, 1996, 2034, 2518, 2008, 22783, 2017

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['for', 'the', 'love', 'of', 'charm', 'what', 'exactly', 'is', 'love', '?', 'do', 'we', 'have', 'to', 'believe', 'in', 'it', 'to', 'be', 'valid', 'lovers', '?', 'signs', 'of', 'primal', 'relationships', 'starbucks', 'tips', 'that', 'imp', '##ote', '##nce', 'is', 'an', 'indicator', 'you', "'", 're', 'engaging', 'in', 'a', 'primal', 'relationship', 'care', '##ment', 'guidance', 'query', 'if', 'you', 'are', 'behind', 'in', 'your', 'care', '##giving', 'and', 'feel', 'like', 'you', "'", 're', 'falling', 'behind', 'partner', 'for', 'partner', 'or', 'the', 'couple', 'is', 'progressing', 'too', 'slow', 'in', 'a', 'relationship', '–', 'check', 'with', 'friends', 'and', 'partners', 'on', 'social', 'media', 'to', 'get', 'suggestions', 'for', 'additional', 'actions', 'to', 'take', 'on', 'a', 'day', '-', 'to', '-', 'day', 'basis', 'bonus', ':', 'trained', 'sex', 'therapist', 'communicate', '##s', 'with', 'partners', 'at', 'length', 'about', 'sexual', 'and', 'relationship', 'satisfaction', 'before',

In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f67fcce81e0>, {'0': 0, '1': 1})


In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)

## Build the Model

首先加载预训练模型

In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

计算模型参数数量，发现参数数量过多，算力不足以训练：

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


由于算力有限，我们对bert中的参数设置为不参与训练

In [ ]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

现在参数量已小于 $3M$, 到达我们可以接受的范围： 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


再次检查模型中的可训练参数，可以发现所有参数都来自GRU (`rnn`) 与全连接层 (`out`).

In [ ]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


## Train the Model


In [ ]:
import torch.optim as optim
import tqdm

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm.tqdm(iterator):
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

训练模型部分，不幸的是，由于colab的连接中断问题，最多的一次也仅**训练3个epoch就发生中断，模型未达收敛**，此时验证集准确率为77.1%，测试集准确率为83.66%。

In [ ]:

N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model-final.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 75m 47s
	Train Loss: 0.476 | Train Acc: 76.01%
	 Val. Loss: 0.507 |  Val. Acc: 76.55%
Epoch: 02 | Epoch Time: 75m 57s
	Train Loss: 0.387 | Train Acc: 81.80%
	 Val. Loss: 0.495 |  Val. Acc: 75.72%
Epoch: 03 | Epoch Time: 75m 58s
	Train Loss: 0.352 | Train Acc: 83.66%
	 Val. Loss: 0.500 |  Val. Acc: 77.10%


## Inference


输出测试集结果

In [ ]:
model.load_state_dict(torch.load('model-final.pt', map_location=torch.device('cpu')))

#test_loss, test_acc = evaluate(model, test_iterator, criterion)

#print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

<All keys matched successfully>

In [ ]:
def predict_isRobot(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_isRobot(model, tokenizer, "I'm not robot!!")

0.6236467957496643

In [ ]:
import tqdm

testFile = open("test1.txt", "r", encoding = 'utf-8')
testoutputFile = open("test1output.txt", "w", encoding = 'utf-8')

for i in tqdm.tqdm(testFile.read().splitlines()):
    print(1 if predict_isRobot(model, tokenizer, i[:-2]) > 0.5 else 0, file=testoutputFile)

testFile2 = open("test2_no_lable.txt", "r", encoding = 'utf-8')
testoutputFile2 = open("test2output.txt", "w", encoding = 'utf-8')

for i in tqdm.tqdm(testFile2.read().splitlines()):
    print(1 if predict_isRobot(model, tokenizer, i[:-2]) > 0.5 else 0, file=testoutputFile2)


验证集上的测试结果见数据分析的notebook.